In [15]:
import requests
from time import sleep

from tqdm import tqdm
import pymongo
from pymongo import MongoClient
import gensim

import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [16]:
import sys
sys.path.append('../')
from src import settings
client = MongoClient(settings.MONGODB_SETTINGS['host'])
db = client[settings.MONGODB_SETTINGS['db']]
col = db[settings.MONGODB_SETTINGS['collection']]

In [17]:
posts = col.find()

In [18]:
col.count_documents({})

176

In [19]:
import joblib
doc_topic_dist = joblib.load(
        settings.PATH_DOC_TOPIC_DIST
    )

In [20]:
def make_sentences():
    for post in col.find():
        yield post['pp_content'] # lấy thuộc tính pp_content của từng document trong post collection
def make_texts_corpus(sentences):
    for sentence in sentences:
        yield simple_preprocess(sentence, deacc=True) # tách chữ cái của từng pp_content
#         yield [word for word in sentence.split()]
from gensim.models.coherencemodel import CoherenceModel
from gensim.utils import simple_preprocess

In [34]:
corpus = gensim.corpora.MmCorpus('../models/corpus.mm')
id2word = gensim.corpora.Dictionary.load(settings.PATH_DICTIONARY)

INFO : loaded corpus index from ../models/corpus.mm.index
INFO : initializing cython corpus reader from ../models/corpus.mm
INFO : accepted corpus with 177 documents, 4087 features, 36185 non-zero entries
INFO : loading Dictionary object from ../models/id2word.dictionary
INFO : Dictionary lifecycle event {'fname': '../models/id2word.dictionary', 'datetime': '2022-12-30T18:32:44.808521', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}


In [35]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, id2word=id2word, num_topics=57, passes=10,
            chunksize=67, random_state=46, alpha=1e-2, eta=0.5e-2,
            minimum_probability=0.0, per_word_topics=False)

INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 57 topics, 10 passes over the supplied corpus of 177 documents, updating model once every 67 documents, evaluating perplexity every 177 documents, iterating 50x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #67/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #46 (0.010): 0.029*"da" + 0.020*"song" + 0.017*"lanh_manh" + 0.016*"loi" + 0.012*"tien" + 0.008*"san_pham" + 0.007*"lan" + 0.007*"benh" + 0.007*"ta" + 0.007*"nghien_cuu"
INFO : topic #38 (0.010): 0.081*"calo" + 0.029*"can" + 0.028*"co_the" + 0.015*"thuc_pham" + 0.014*"nang_luong" + 0.013*"nap" + 0.012*"song" + 0.011*"da" + 0.010*"loi" + 0.009*"chay"
INFO : topic #51 (0.010): 0.019*"khoi_nghiep" + 0.018*"kinh_doanh" + 0.011*"song" + 0.009*"calo" + 0.008*"loi" + 0.007*"lam_viec" + 0.007*"thong_minh" + 0.006*"can" + 0.006*"lanh_manh" + 0.006*"mua"
INFO : topic #5 (0.010):

INFO : topic #51 (0.010): 0.132*"khoi_nghiep" + 0.107*"kinh_doanh" + 0.037*"y_tuong" + 0.035*"thanh_cong" + 0.032*"ban_tay_trang" + 0.025*"von" + 0.020*"giau" + 0.019*"khach_hang" + 0.017*"cong_viec" + 0.016*"ban_hoa"
INFO : topic diff=8.337376, rho=0.421009
INFO : PROGRESS: pass 2, at document #134/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #28 (0.010): 0.391*"khung_hoang" + 0.191*"kinh_te" + 0.082*"bong_bong" + 0.020*"lam_phat" + 0.013*"kinh_doanh" + 0.013*"thi_truong" + 0.012*"giay_to" + 0.008*"tai_chinh" + 0.006*"mo_rong" + 0.004*"khung"
INFO : topic #47 (0.010): 0.051*"du_lich" + 0.050*"van_hoa" + 0.031*"thuoc" + 0.022*"đep" + 0.020*"mien" + 0.020*"truyen_thong" + 0.019*"phong_tuc" + 0.019*"may_man" + 0.019*"tay" + 0.018*"bac"
INFO : topic #0 (0.010): 0.028*"thuc_pham" + 0.028*"che_đo" + 0.022*"chat" + 0.019*"an_uong" + 0.018*"suc_khoe" + 0.017*"co_the" + 0.016*"benh" + 0.016*"lanh_manh" + 0.015*"chua" + 0.015*"uong"
INFO : topic #2 (0.

INFO : topic diff=1.693800, rho=0.361745
INFO : -33.414 per-word bound, 11443873341.3 perplexity estimate based on a held-out corpus of 43 documents with 22947 words
INFO : PROGRESS: pass 4, at document #177/177
INFO : merging changes from 43 documents into a model of 177 documents
INFO : topic #51 (0.010): 0.135*"khoi_nghiep" + 0.103*"kinh_doanh" + 0.034*"thanh_cong" + 0.034*"y_tuong" + 0.026*"khach_hang" + 0.026*"tu_hao" + 0.025*"von" + 0.024*"ban_tay_trang" + 0.021*"cong_viec" + 0.021*"tom_tat"
INFO : topic #19 (0.010): 0.000*"marketing" + 0.000*"chuoi" + 0.000*"kinh_doanh" + 0.000*"khoi_nghiep" + 0.000*"mua" + 0.000*"lam_viec" + 0.000*"thanh_cong" + 0.000*"sprint" + 0.000*"agile" + 0.000*"san_pham"
INFO : topic #34 (0.010): 0.047*"web" + 0.034*"html" + 0.033*"trang_web" + 0.033*"thiet_ke" + 0.030*"javascript" + 0.030*"de_dang" + 0.026*"vu" + 0.023*"chuong_trinh" + 0.021*"ui" + 0.020*"ma"
INFO : topic #27 (0.010): 0.091*"phong" + 0.083*"thue" + 0.072*"tro" + 0.030*"xe" + 0.026*"tien

INFO : topic diff=0.473952, rho=0.322048
INFO : PROGRESS: pass 7, at document #67/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #19 (0.010): 0.000*"marketing" + 0.000*"chuoi" + 0.000*"kinh_doanh" + 0.000*"khoi_nghiep" + 0.000*"mua" + 0.000*"lam_viec" + 0.000*"thanh_cong" + 0.000*"sprint" + 0.000*"agile" + 0.000*"san_pham"
INFO : topic #54 (0.010): 0.031*"ung_dung" + 0.021*"cong_nghiep" + 0.019*"vr" + 0.018*"cong_nghe" + 0.017*"thiet_bi" + 0.014*"doanh_nghiep" + 0.012*"he_thong" + 0.011*"xay_dung" + 0.011*"du_lieu" + 0.010*"trai"
INFO : topic #5 (0.010): 0.050*"kinh_doanh" + 0.043*"khoi_nghiep" + 0.039*"huong" + 0.035*"con_so" + 0.031*"chu_đao" + 0.028*"quy" + 0.023*"muc_tieu" + 0.021*"cong_ty" + 0.018*"so_huu" + 0.016*"mao_hiem"
INFO : topic #28 (0.010): 0.428*"khung_hoang" + 0.383*"kinh_te" + 0.067*"bong_bong" + 0.023*"lam_phat" + 0.009*"kinh_doanh" + 0.008*"giay_to" + 0.008*"thi_truong" + 0.004*"mo_rong" + 0.002*"tai_chinh" + 0.000*"khung"
IN

INFO : topic diff=0.309698, rho=0.281252
INFO : PROGRESS: pass 9, at document #134/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #29 (0.010): 0.275*"tay" + 0.242*"chet" + 0.053*"ban" + 0.052*"cang" + 0.048*"yen" + 0.041*"trang" + 0.024*"khu_vuc" + 0.018*"vien" + 0.018*"san_pham" + 0.017*"khoa_hoc"
INFO : topic #52 (0.010): 0.469*"lang" + 0.090*"đong_y" + 0.024*"chong_mat" + 0.001*"tri" + 0.000*"ngu" + 0.000*"giac_ngu" + 0.000*"da" + 0.000*"ngon" + 0.000*"giac" + 0.000*"thai"
INFO : topic #24 (0.010): 0.000*"da" + 0.000*"he_thong" + 0.000*"du_lieu" + 0.000*"tang_truong" + 0.000*"thong_bao" + 0.000*"du_bao" + 0.000*"website" + 0.000*"blog" + 0.000*"phan_tu" + 0.000*"kinh_te"
INFO : topic #45 (0.010): 0.378*"ngon_ngu" + 0.190*"lap_trinh" + 0.090*"java" + 0.047*"swift" + 0.044*"pho_bien" + 0.043*"php" + 0.037*"javascript" + 0.032*"python" + 0.028*"ung_dung" + 0.024*"chay"
INFO : topic #39 (0.010): 0.199*"loc" + 0.100*"xanh" + 0.093*"mua" + 0.066*"b

In [36]:
sentences = make_sentences()
sentences = make_texts_corpus(sentences)
coherence_model_lda = CoherenceModel(model=lda_model, texts=sentences, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

INFO : using ParallelWordOccurrenceAccumulator<processes=7, batch_size=64> to estimate probabilities from sliding windows
INFO : 1 batches submitted to accumulate stats from 64 documents (25378 virtual)
INFO : 2 batches submitted to accumulate stats from 128 documents (49615 virtual)
INFO : 3 batches submitted to accumulate stats from 192 documents (72459 virtual)
INFO : 7 accumulators retrieved from output queue
INFO : accumulated word occurrence stats for 72512 virtual documents



Coherence Score:  0.4930004675647774


In [37]:
import gensim
corpus = gensim.corpora.MmCorpus('../models/corpus.mm')
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

INFO : loaded corpus index from ../models/corpus.mm.index
INFO : initializing cython corpus reader from ../models/corpus.mm
INFO : accepted corpus with 177 documents, 4087 features, 36185 non-zero entries
INFO : -22.623 per-word bound, 6457650.2 perplexity estimate based on a held-out corpus of 177 documents with 81947 words



Perplexity:  -22.62257786771165


In [25]:
def evaluate_quality_model(num_topics, chunksize):
    lda_model = gensim.models.ldamodel.LdaModel(corpus, id2word=id2word, num_topics=num_topics, passes=10,
            chunksize=chunksize, random_state=46, alpha=0.01, eta=0.005,
            minimum_probability=0.0, per_word_topics=False)
    sentences = make_sentences()
    sentences = make_texts_corpus(sentences)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=sentences, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    perplexity = lda_model.log_perplexity(corpus)
    return coherence_lda, perplexity

In [26]:
list = []
for num_topics in range(55, 90):
    coherence_lda, perplexity = evaluate_quality_model(num_topics, 67)
    list.append((num_topics, coherence_lda, perplexity))

INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 55 topics, 10 passes over the supplied corpus of 177 documents, updating model once every 67 documents, evaluating perplexity every 177 documents, iterating 50x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #67/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #2 (0.010): 0.010*"user" + 0.009*"stories" + 0.009*"du_lich" + 0.008*"san_pham" + 0.008*"da" + 0.007*"kinh_doanh" + 0.007*"chuyen" + 0.007*"khach_hang" + 0.007*"thong_minh" + 0.006*"song"
INFO : topic #16 (0.010): 0.021*"marketing" + 0.011*"agile" + 0.011*"mo_hinh" + 0.011*"ung_dung" + 0.010*"ngon_ngu" + 0.009*"swift" + 0.008*"du_lich" + 0.007*"khoi_nghiep" + 0.007*"may" + 0.007*"hoc"
INFO : topic #37 (0.010): 0.014*"du_lich" + 0.010*"bui" + 0.008*"du_lieu" + 0.007*"thong_tin" + 0.007*"api" + 0.007*"an_uong" + 0.006*"lanh_manh" + 0.006*"he_thong" + 0.006*"khoi_nghiep" 

INFO : topic diff=8.306776, rho=0.421009
INFO : PROGRESS: pass 2, at document #134/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #18 (0.010): 0.225*"website" + 0.090*"blog" + 0.052*"case" + 0.048*"binh_luan" + 0.042*"topic" + 0.035*"đanh_dau" + 0.031*"chinh_sua" + 0.030*"thiet_ke" + 0.027*"chu_đe" + 0.021*"theo_doi"
INFO : topic #17 (0.010): 0.008*"calo" + 0.007*"da" + 0.003*"can" + 0.003*"co_the" + 0.003*"skincare" + 0.002*"thuc_pham" + 0.002*"nang_luong" + 0.002*"mat" + 0.002*"cham_soc" + 0.002*"chat"
INFO : topic #26 (0.010): 0.072*"phanh" + 0.072*"nop" + 0.070*"sang_loc" + 0.065*"down" + 0.048*"bitcoin" + 0.037*"sa_thai" + 0.024*"doc" + 0.023*"khuon_mat" + 0.018*"nhan_su" + 0.017*"khung_hoang"
INFO : topic #37 (0.010): 0.217*"nha" + 0.007*"du_lich" + 0.002*"du_khach" + 0.002*"bui" + 0.001*"du_lieu" + 0.001*"thong_tin" + 0.001*"kham_pha" + 0.001*"api" + 0.001*"goi_y" + 0.001*"an_uong"
INFO : topic #12 (0.010): 0.003*"khoi_nghiep" + 0.003*"th

INFO : topic diff=1.740440, rho=0.361745
INFO : -33.478 per-word bound, 11961335283.5 perplexity estimate based on a held-out corpus of 43 documents with 22947 words
INFO : PROGRESS: pass 4, at document #177/177
INFO : merging changes from 43 documents into a model of 177 documents
INFO : topic #43 (0.010): 0.074*"may" + 0.068*"san_xuat" + 0.066*"cong_nghe" + 0.054*"he_thong" + 0.053*"nhan_tao" + 0.047*"con_nguoi" + 0.041*"tri_tue" + 0.035*"tu_đong" + 0.032*"thong_minh" + 0.027*"kha_nang"
INFO : topic #52 (0.010): 0.082*"đam_bao_ve_sinh" + 0.001*"giac_ngu" + 0.001*"ngu" + 0.001*"da" + 0.000*"ngon" + 0.000*"gian" + 0.000*"co_the" + 0.000*"nao" + 0.000*"suc_khoe" + 0.000*"kinh_te"
INFO : topic #9 (0.010): 0.000*"lai_suat" + 0.000*"kinh_doanh" + 0.000*"khoi_nghiep" + 0.000*"thi_truong" + 0.000*"kinh_te" + 0.000*"thong_minh" + 0.000*"du_bao" + 0.000*"may" + 0.000*"au" + 0.000*"he_thong"
INFO : topic #51 (0.010): 0.194*"cung" + 0.129*"mam" + 0.081*"nhan_xet" + 0.043*"gia_đinh" + 0.032*"đong

INFO : topic #11 (0.010): 0.052*"du_lieu" + 0.052*"code" + 0.048*"javascript" + 0.037*"html" + 0.027*"web" + 0.026*"thu_vien" + 0.025*"đon_gian" + 0.024*"loi" + 0.023*"giao_dien" + 0.023*"framework"
INFO : topic diff=0.468760, rho=0.322048
INFO : PROGRESS: pass 7, at document #67/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #42 (0.010): 0.000*"phan_tu" + 0.000*"java" + 0.000*"ngon_ngu" + 0.000*"ung_dung" + 0.000*"sap_xep" + 0.000*"calo" + 0.000*"lap_trinh" + 0.000*"noi" + 0.000*"hoc" + 0.000*"vong"
INFO : topic #21 (0.010): 0.164*"api" + 0.123*"ta" + 0.090*"request" + 0.051*"view" + 0.035*"model" + 0.029*"response" + 0.023*"database" + 0.021*"fed" + 0.020*"that_lung" + 0.020*"token"
INFO : topic #4 (0.010): 0.045*"tien" + 0.024*"đau_tu" + 0.014*"sinh_vien" + 0.014*"muc_tieu" + 0.010*"he" + 0.010*"ngan_hang" + 0.010*"vi_du" + 0.009*"chuyen_đoi" + 0.009*"chi_phi" + 0.008*"tro_nen"
INFO : topic #3 (0.010): 0.037*"kinh_te" + 0.024*"chi_tieu" + 0.0

INFO : topic #11 (0.010): 0.058*"du_lieu" + 0.045*"code" + 0.035*"javascript" + 0.029*"html" + 0.024*"đon_gian" + 0.023*"backend" + 0.023*"loi" + 0.022*"giao_dien" + 0.022*"chay" + 0.022*"ma"
INFO : topic diff=0.321731, rho=0.281252
INFO : PROGRESS: pass 9, at document #134/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #23 (0.010): 0.085*"loc" + 0.071*"khong_khi" + 0.056*"may" + 0.030*"bui" + 0.021*"sach" + 0.017*"mui" + 0.014*"san_pham" + 0.014*"tho" + 0.012*"khong_gian" + 0.012*"xanh"
INFO : topic #52 (0.010): 0.123*"đam_bao_ve_sinh" + 0.000*"giac_ngu" + 0.000*"ngu" + 0.000*"da" + 0.000*"ngon" + 0.000*"gian" + 0.000*"co_the" + 0.000*"nao" + 0.000*"suc_khoe" + 0.000*"kinh_te"
INFO : topic #45 (0.010): 0.000*"ngon_ngu" + 0.000*"lap_trinh" + 0.000*"ung_dung" + 0.000*"java" + 0.000*"swift" + 0.000*"thong_minh" + 0.000*"calo" + 0.000*"con_nguoi" + 0.000*"chay" + 0.000*"pho_bien"
INFO : topic #22 (0.010): 0.053*"vr" + 0.029*"san_pham" + 0.024*"mat"

...

INFO : topic #10 (0.010): 0.614*"ao" + 0.120*"ta" + 0.105*"nam" + 0.046*"hai" + 0.020*"thoi_ky" + 0.018*"lop" + 0.016*"vay" + 0.010*"tai_lieu" + 0.009*"dang" + 0.005*"co_quan"
INFO : topic diff=0.214870, rho=0.293083
INFO : PROGRESS: pass 9, at document #67/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #33 (0.010): 0.107*"tay" + 0.025*"đep" + 0.021*"kho" + 0.020*"cang" + 0.019*"ban" + 0.018*"bo_qua" + 0.018*"phat_huy" + 0.018*"kem" + 0.016*"long" + 0.016*"bui"
INFO : topic #66 (0.010): 0.088*"ky_su" + 0.058*"cong_ty" + 0.056*"ruby" + 0.056*"github" + 0.050*"it" + 0.043*"thinh_hanh" + 0.040*"go" + 0.040*"hieu_nang" + 0.037*"iphone" + 0.035*"swift"
INFO : topic #51 (0.010): 0.328*"met_moi" + 0.099*"ue_oai" + 0.017*"so_sanh" + 0.000*"song" + 0.000*"khoi_nghiep" + 0.000*"kinh_doanh" + 0.000*"loi" + 0.000*"phan_tu" + 0.000*"lanh_manh" + 0.000*"nghien_cuu"
INFO : topic #75 (0.010): 0.000*"lanh_manh" + 0.000*"song" + 0.000*"suc_khoe" + 0.000*"tien" + 

INFO : topic #61 (0.010): 0.029*"thi_điem" + 0.012*"đoi" + 0.010*"agile" + 0.009*"he_thong" + 0.008*"thong_minh" + 0.007*"xay_dung" + 0.007*"vong" + 0.007*"mo_hinh" + 0.006*"kha_nang" + 0.006*"muc_tieu"
INFO : topic diff=9.928782, rho=0.577350
INFO : PROGRESS: pass 1, at document #67/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #60 (0.010): 0.110*"ngon_ngu" + 0.041*"đien" + 0.035*"lap_trinh" + 0.024*"kiem_tra" + 0.022*"kenh" + 0.019*"mu" + 0.019*"đanh_dau" + 0.016*"java" + 0.014*"theo_doi" + 0.014*"van_ban"
INFO : topic #45 (0.010): 0.021*"ngon_ngu" + 0.015*"calo" + 0.013*"lap_trinh" + 0.008*"co_the" + 0.008*"can" + 0.005*"ung_dung" + 0.005*"thuc_pham" + 0.005*"java" + 0.004*"kcal" + 0.004*"nap"
INFO : topic #36 (0.010): 0.042*"backend" + 0.031*"frontend" + 0.030*"kinh_doanh" + 0.023*"smartphone" + 0.022*"danh_sach" + 0.020*"thong_minh" + 0.018*"quan_ly" + 0.017*"khoi_nghiep" + 0.016*"thanh_cong" + 0.016*"lam_viec"
INFO : topic #2 (0.010): 0.0

INFO : topic #80 (0.010): 0.001*"kinh_te" + 0.001*"da" + 0.001*"khung_hoang" + 0.001*"tui" + 0.001*"backend" + 0.001*"ta" + 0.001*"lan" + 0.001*"file" + 0.001*"ti_le" + 0.000*"tai_chinh"
INFO : topic diff=3.667453, rho=0.388023
INFO : PROGRESS: pass 3, at document #134/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #48 (0.010): 0.001*"thong_minh" + 0.001*"song" + 0.001*"lanh_manh" + 0.001*"material" + 0.001*"kinh_doanh" + 0.001*"khoi_nghiep" + 0.001*"loi" + 0.001*"design" + 0.001*"ta" + 0.001*"thiet_ke"
INFO : topic #76 (0.010): 0.065*"che_đo" + 0.061*"an_uong" + 0.046*"lanh_manh" + 0.041*"chat" + 0.038*"benh" + 0.031*"thuc_pham" + 0.031*"rau" + 0.029*"suc_khoe" + 0.028*"ngu_coc" + 0.027*"trai_cay"
INFO : topic #16 (0.010): 0.124*"agile" + 0.120*"marketing" + 0.051*"swift" + 0.044*"tuan" + 0.032*"mo_hinh" + 0.032*"thuong_xuyen" + 0.027*"bo_nhiem" + 0.025*"phan_mem" + 0.025*"theo_doi" + 0.024*"planning"
INFO : topic #0 (0.010): 0.041*"xanh" + 0.0

INFO : topic diff=0.546415, rho=0.340172
INFO : -33.488 per-word bound, 12046557766.6 perplexity estimate based on a held-out corpus of 43 documents with 22947 words
INFO : PROGRESS: pass 5, at document #177/177
INFO : merging changes from 43 documents into a model of 177 documents
INFO : topic #16 (0.010): 0.178*"agile" + 0.070*"marketing" + 0.055*"bo_nhiem" + 0.053*"mo_hinh" + 0.040*"tuan" + 0.038*"phan_mem" + 0.032*"swift" + 0.032*"đoi" + 0.028*"thuong_xuyen" + 0.025*"planning"
INFO : topic #82 (0.010): 0.219*"vu" + 0.165*"vat" + 0.070*"tuong_đuong" + 0.068*"hoan_thanh" + 0.030*"tac" + 0.028*"lop" + 0.026*"cho" + 0.026*"so_luong" + 0.023*"chuong_trinh" + 0.023*"phuong_thuc"
INFO : topic #56 (0.010): 0.154*"nam" + 0.138*"ta" + 0.130*"viet_nam" + 0.105*"quoc_te" + 0.081*"quoc_gia" + 0.066*"khi_hau" + 0.055*"kinh_te" + 0.045*"the_gioi" + 0.043*"bien_đoi" + 0.017*"khu_vuc"
INFO : topic #24 (0.010): 0.000*"phan_tu" + 0.000*"da" + 0.000*"sap_xep" + 0.000*"vong" + 0.000*"bot" + 0.000*"noi"

INFO : topic diff=0.236488, rho=0.306544
INFO : PROGRESS: pass 8, at document #67/177
INFO : merging changes from 67 documents into a model of 177 documents
INFO : topic #15 (0.010): 0.000*"react" + 0.000*"dom" + 0.000*"khoi_nghiep" + 0.000*"tien" + 0.000*"logic" + 0.000*"sach" + 0.000*"y_tuong" + 0.000*"cap_nhat" + 0.000*"kinh_doanh" + 0.000*"goi"
INFO : topic #59 (0.010): 0.000*"da" + 0.000*"skincare" + 0.000*"lan" + 0.000*"mat" + 0.000*"duong" + 0.000*"cham_soc" + 0.000*"quy_trinh" + 0.000*"am" + 0.000*"kem" + 0.000*"san_pham"
INFO : topic #40 (0.010): 0.000*"chuoi" + 0.000*"kinh_te" + 0.000*"is" + 0.000*"this" + 0.000*"khung_hoang" + 0.000*"nhap" + 0.000*"the_gioi" + 0.000*"tien" + 0.000*"mot" + 0.000*"đau_tu"
INFO : topic #85 (0.010): 0.015*"cuoc_song" + 0.013*"than" + 0.013*"song" + 0.011*"con_nguoi" + 0.007*"hai" + 0.007*"cong_viec" + 0.007*"co" + 0.007*"suc_khoe" + 0.007*"phu_nu" + 0.007*"thoi_quen"
INFO : topic #69 (0.010): 0.074*"chu_tich" + 0.058*"an_ninh" + 0.054*"thu_tuong

In [27]:
list

[(55, 0.45199761038348163, -22.682007947308424),
 (56, 0.48173630536965184, -22.6271606695987),
 (57, 0.4930004675647774, -22.62257786771165),
 (58, 0.5034202449517065, -22.951636135437777),
 (59, 0.45794142589344794, -22.813615620888346),
 (60, 0.4832722244399653, -22.776008899603752),
 (61, 0.484335797546654, -22.645536622721853),
 (62, 0.47155618208289724, -22.61633228473914),
 (63, 0.4667592838693721, -22.87137325332677),
 (64, 0.4510306670465831, -23.00824182882538),
 (65, 0.4668730303942429, -22.93367533679078),
 (66, 0.4849612410617583, -22.87743539335261),
 (67, 0.4691060672552515, -22.833234272062146),
 (68, 0.4519531741994433, -22.76463678551603),
 (69, 0.4720124568726547, -22.58381492267169),
 (70, 0.47976189506898775, -22.592791634391386),
 (71, 0.47404261545093274, -22.82787260460048),
 (72, 0.48110114661133196, -22.6223340727597),
 (73, 0.4668439213969381, -23.00068987521153),
 (74, 0.461754475465016, -22.590626881554115),
 (75, 0.4501577492359644, -22.890704352174122),
 